In [78]:
import numpy as np
import scipy.constants
import math
import re
from astropy import constants as const

In [79]:
class Vector:
    def __init__(self, x0, x1, x2):
        self.array = np.array([x0, x1, x2])
    
    def length(self):
        return np.linalg.norm(self.array)

In [80]:
class QuantityDimension:
    def __init__(self, L=0, M=0, T=0, I=0, Θ=0, N=0, J=0):
        self.L = L
        self.M = M
        self.T = T
        self.I = I
        self.Θ = Θ
        self.J = J
 
    def getDimensions():
        return M

In [567]:
class QuantityValue:
    def __init__(self, number, reference, standardUncertainty):
        self.number = number
        self.reference = reference
        self.standardUncertainty = abs(standardUncertainty)
        self.relativeUncertainty = self.standardUncertainty / abs(self.number)
        
        self.referenceList = [re.split('\^', unit) for unit in re.split('\s+', self.reference)]
        self.dimList = filter(lambda i: (i[0] != '1' and i[0] != ''), [[dim[0], int(dim[1]) if len(dim) == 2 else 1] for dim in self.referenceList])
        self.dimDict = {}
        for i in self.dimList:
            self.dimDict[i[0]] = (self.dimDict[i[0]] + i[1]) if i[0] in self.dimDict else i[1]
        self.reference = ''
        for key, exp in list(self.dimDict.items()):
            if exp == 0:
                del self.dimDict[key]
                continue
            self.reference += '{}{}{}'.format(
                '' if self.reference == '' else ' ',
                key if exp !=0 else '',
                '^' + str(exp) if (exp != 0 and exp != 1) else '')
                               
    def __str__(self):
        exponent = math.floor(math.log10(abs(self.number)))
        engExponent = math.floor(exponent / 3) * 3
        engMantissa = self.number / 10**engExponent
        if self.standardUncertainty != 0:
            uncertaintyExponent = math.floor(math.log10(self.standardUncertainty))
            uncertaintyMantissa = self.standardUncertainty / 10**(uncertaintyExponent-1)
            numDigits = engExponent - uncertaintyExponent + 1
            uncertaintyStr = '(' + '{:.0f}'.format(uncertaintyMantissa) + ')'
        else:
            numDigits = '4'
            uncertaintyStr = '(...)'
        if engExponent != 0:
            exponentStr = 'e' + repr(engExponent)
        else:
            exponentStr = ""
        return '{:.{}f}'.format(engMantissa, numDigits) + uncertaintyStr + exponentStr + ' ' + self.reference
    
    def __repr__(self):
        return self.__str__()
    
    def invReference(self):
        referenceStr = ''
        for key in self.dimDict:
            exp = -1 * self.dimDict[key]
            referenceStr += '{}{}{}'.format(
                '' if referenceStr == '' else ' ',
                key if exp != 0 else '',
                '^' + str(exp) if (exp != 0 and exp != 1) else ''
            )
        return referenceStr
    
    def exp(self):
        if self.dimDict == {}:
            return QuantityValue(math.exp(self.number), '', math.exp(self.number) * self.standardUncertainty)
        else:
            raise TypeError("exp function only applies to quantities of dimension 1")
    
    def __add__(self, other):
        if isinstance(other, self.__class__) and self.dimDict == other.dimDict:
            return QuantityValue(self.number + other.number,
                                 self.reference,
                                 math.sqrt(self.standardUncertainty**2 + other.standardUncertainty**2))
        elif isinstance(other, (int, float)) and self.dimDict == {}:
            return QuantityValue(other + self.number,
                             self.reference,
                             abs(other) * self.standardUncertainty)
        else:
            raise TypeError("cannot add values with differint quantity dimensions")
    
    def __radd__(self, other):
        return self.__add__(other)
    
    def __sub__(self,other):
        if isinstance(other, self.__class__) and self.dimDict == other.dimDict:
            return QuantityValue(self.number - other.number,
                                 self.reference,
                                 math.sqrt(self.standardUncertainty**2 + other.standardUncertainty**2))
        else:
            raise TypeError("cannot subtract values with differint quantity dimensions")
                               
    def __mul__(self, other):
        if isinstance(other, self.__class__):
            return QuantityValue(self.number * other.number,
                                 self.reference + ' ' + other.reference,
                                 abs(self.number * other.number) 
                                 * math.sqrt(self.relativeUncertainty**2 + other.relativeUncertainty**2))
        elif isinstance(other, (int, float)):
            return QuantityValue(other * self.number,
                             self.reference,
                             abs(other) * self.standardUncertainty)
        else:
            raise TypeError("types not supported for multiplication")
    
    def __rmul__(self, other):
        return self.__mul__(other)
                        
    def __truediv__(self, other):
        if isinstance(other, self.__class__):
            return QuantityValue(self.number / other.number,
                                 self.reference + ' ' + other.invReference(),
                                 abs(self.number / other.number) 
                                 * math.sqrt(self.relativeUncertainty**2 + other.relativeUncertainty**2))
        elif isinstance(other, (int, float)):
            return QuantityValue(self.number / other,
                             self.reference,
                             abs(1 / other) * self.standardUncertainty)
    
    def __rtruediv__(self, other):
        if isinstance(other, (int, float)):
            return QuantityValue(other / self.number,
                             self.reference,
                             abs(other) * math.sqrt(self.relativeUncertainty**2))
    
    def __neg__(self):
        return QuantityValue(-1 * self.number,
                             self.reference,
                             self.standardUncertainty)

In [568]:
def getScipyConstant(key):
    return QuantityValue(scipy.constants.value(key),
                  scipy.constants.unit(key),
                  scipy.constants.precision(key) * abs(scipy.constants.value(key)))

In [569]:
pi = scipy.constants.pi
G = getScipyConstant('Newtonian constant of gravitation')
c = getScipyConstant('speed of light in vacuum')
g = getScipyConstant('standard acceleration of gravity')

In [584]:
linearDensity = QuantityValue(0.01, 'kg m^-1', 0)
angle = QuantityValue(0.25, '1', 0)
altitude = QuantityValue(300E3, 'm', 0)
R_Earth = QuantityValue(6378137, 'm', 2) #WGS84
M_Earth = QuantityValue(5.972E24, 'm', 0)
payload = QuantityValue(1E3, 'kg', 0)
tensileStrength = QuantityValue(3.4E9, 'kg m s^-2 m^-2', 0)
massFlowRate = 1 # kg/s
massDepositionEfficiency = .5
speed = QuantityValue(math.sqrt((G * M_Earth / (R_Earth + altitude)).number), 'm s^-1', 0)
ladderDensity = QuantityValue(1.5e3, 'kg m^-3', 0)

In [585]:
speed

7.7255(...)e3 m s^-1

In [586]:
hookLift = linearDensity * speed * speed * angle
print(hookLift)

149.2093(...)e3 kg s^-2 m


In [587]:
(R_Earth + altitude) * linearDensity * 2 * math.pi / QuantityValue(64000, 'kg (launches)^-1', 0)

6.5562457(20) (launches)

In [588]:
def areaTop(payload, density, tensileStrength, altitude):
    return payload * g / tensileStrength * (density * g / tensileStrength * altitude / (1 + altitude / R_Earth)).exp()

In [589]:
area = areaTop(payload, ladderDensity, tensileStrength, altitude)

In [590]:
area

9.96338957(17)e-6 m^2

In [591]:
def forceTop(payload, density, tensileStrength, altitude):
    return payload * g * (density * g / tensileStrength * altitude / (1 + altitude / R_Earth)).exp()

In [592]:
force = forceTop(payload, ladderDensity, tensileStrength, altitude)

In [599]:
force

33.87552455(59)e3 kg m s^-2

In [600]:
P = (-ladderDensity * g / tensileStrength * altitude / (1 + altitude / R_Earth)).exp()

In [601]:
1 / P

3.454342160(17) 

In [602]:
hookLift * P / g

4.404634365(77)e3 kg